In [24]:
import pandas as pd
import re
import nltk
import json
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
import emoji
import unicodedata

In [25]:
df = pd.read_csv('data.csv')
df


,full_text,case_folding CPMK2,preprocessing CPMK3,label
0,@CakKhum Ya Smga ALLÀH dg Seluruh Kekuatan ALA...,@cakkhum ya smga allàh dg seluruh kekuatan ala...,smga all h kuat alam nya jujur amp dahsyat kua...,positive
1,@Nexus031191 @puzzwles jangan ada wamen2 yang ...,@nexus031191 @puzzwles jangan ada wamen2 yang ...,wamen angkat pres susah ngaturnya klau dirjen ...,positive
2,@sharpandshark mentri purbaya uang harus dipak...,@sharpandshark mentri purbaya uang harus dipak...,tri uang pakai putar gerak ekonomi masyarakat ...,positive
3,@sharpandshark Beda dengan mentri purbaya sang...,@sharpandshark beda dengan mentri purbaya sang...,beda tri sang tri tri uang rida ekonomi nasion...,positive
4,new idol has been unlock Purbaya Yudhi Sadewaa...,new idol has been unlock purbaya yudhi sadewaa...,new idol has been unlock yudhi sadewa,positive
...,...,...,...,...
245,@AnKiiim_ Waham kabeh Dokteeeeeerrr ini ada pa...,@ankiiim_ waham kabeh dokteeeeeerrr ini ada pa...,waham kabeh dokter pasien kabuurr gt,negative
246,Keren semoga bukan omon² nantinya ya pak.. Ins...,keren semoga bukan omon² nantinya ya pak.. ins...,keren moga omon insyaallah kerja hasil tunggu ...,negative
247,Heboh! Anak Menkeu Purbaya Sadewa Pamer Gepoka...,heboh! anak menkeu purbaya sadewa pamer gepoka...,heboh anak sadewa pamer gepok dolar as hasil t...,negative
248,Ngebuzzer Purbaya dulu yak :) https://t.co/Za6...,ngebuzzer purbaya dulu yak :) https://t.co/za6...,ngebuzzer yak,negative


In [26]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
stop_words = set(stopwords.words("indonesian"))

stopwords_extra = {
    "nya", "sih", "saja", "aja", "kok", "sama", "pun", "lah", "dong", "deh",
    "banget", "bgt", "yg", "ya", "kan", "kah", "nah", "loh", "tuh", "nih",
    "gitu", "gituan", "begitu", "oke", "ok", "yaa", "yaaah", "yah", "iyaa",
    "iya", "engga", "nggak", "ngga", "gak", 'amp', 'all', 'h', 'gt', 'yak', 'rida','tri'
}

stop_words.update(stopwords_extra)

import json
with open("dopping.json", "r", encoding="utf-8") as f:
    abbreviation_dict = json.load(f)

bentuk_kata = re.compile(r'^[a-z]+$')

def reduce_repeated_chars(word):
    return re.sub(r'(.)\1{2,}', r'\1', word)

def remove_non_latin(text):
    return ''.join(
        c for c in text 
        if unicodedata.category(c)[0] != 'C'
        and "LATIN" in unicodedata.name(c, 'LATIN')
    )

def preprocess(text):
    if pd.isna(text):
        return ""

def remove_non_latin(text):
    return re.sub(r'[^\x00-\x7F]+', ' ', text)  

def preprocess(text):
    if pd.isna(text):
        return ""

    text = text.lower()
    text = re.sub(r'http\S+|www\S+', ' ', text)
    text = emoji.replace_emoji(text, replace=' ')
    text = re.sub(r'[@#]\w+', ' ', text)

    text = remove_non_latin(text)

    text = re.sub(r'[^a-z\s]', ' ', text)

    words = text.split()
    cleaned = []

    for w in words:
        w = reduce_repeated_chars(w)

        if w in abbreviation_dict:
            w = abbreviation_dict[w]

        if not bentuk_kata.match(w):
            continue

        if w in stop_words:
            continue

        w = stemmer.stem(w)

        if w in stop_words:
            continue

        cleaned.append(w)

    cleaned_text = " ".join(cleaned)
    return cleaned_text


In [27]:
df["bersih"] = df["preprocessing CPMK3"].apply(preprocess)
df['bersih']

0      smga kuat alam jujur dahsyat kuat dahsyat muda...
1      wamen angkat pres susah ngaturnya klau dirjen ...
2      uang pakai putar gerak ekonomi masyarakat simp...
3      beda sang uang ekonomi nasional lambat lesu in...
4                  new idol has been unlock yudhi sadewa
                             ...                        
245                    waham kabeh dokter pasien kabuurr
246    keren moga omon kerja hasil tunggu aksi pajak ...
247    heboh anak sadewa pamer gepok dolar as hasil t...
248                                            ngebuzzer
249    tolak halus program langgar kode etik bisnis s...
Name: bersih, Length: 250, dtype: object

<h1>1. SVM + TF-IDF</h1>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score

X = df["bersih"]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1,2)
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


scenarios = [
    {"name": "C=0.1", "params": {"C": 0.1}},
    {"name": "C=1",   "params": {"C": 1}},
    {"name": "C=10",  "params": {"C": 10}},
    {"name": "C=100", "params": {"C": 100}},
    {"name": "C=1 + balanced", "params": {"C": 1, "class_weight": "balanced"}},
]

results = []
best_model = None
best_acc = 0
best_name = ""


for sc in scenarios:
    print(f"\n======================")
    print(f"Running: {sc['name']}")
    print("======================")
    
    model = LinearSVC(**sc["params"])
    model.fit(X_train_tfidf, y_train)
    
    pred = model.predict(X_test_tfidf)
    acc = accuracy_score(y_test, pred)
    print(f"Akurasi: {acc:.4f}")
    
    results.append((sc["name"], acc))
    
    if acc > best_acc:
        best_acc = acc
        best_model = model
        best_name = sc["name"]
        best_pred = pred

print("\n======================================")
print("HASIL TERBAIK")
print("======================================")
print(f"Skenario terbaik : {best_name}")
print(f"Akurasi terbaik  : {best_acc:.4f}\n")

print("Classification Report (BEST MODEL):")
print(classification_report(y_test, best_pred))



Running: C=0.1
Akurasi: 0.6000

Running: C=1
Akurasi: 0.5600

Running: C=10
Akurasi: 0.5200

Running: C=100
Akurasi: 0.5400

Running: C=1 + balanced
Akurasi: 0.5400

HASIL TERBAIK
Skenario terbaik : C=0.1
Akurasi terbaik  : 0.6000

Classification Report (BEST MODEL):
              precision    recall  f1-score   support

    negative       0.55      0.57      0.56        21
     neutral       0.60      0.43      0.50         7
    positive       0.65      0.68      0.67        22

    accuracy                           0.60        50
   macro avg       0.60      0.56      0.57        50
weighted avg       0.60      0.60      0.60        50



C:\Users\ASUS\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\ASUS\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\ASUS\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\ASUS\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  war

<h1>2. SVM + Word2Vec</h1>

In [ ]:
from gensim.models import Word2Vec
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

sentences = df['bersih'].apply(lambda x: x.split())
w2v = Word2Vec(sentences, vector_size=200, min_count=2, window=5)

def get_w2v_vector(words):
    vecs = [w2v.wv[w] for w in words if w in w2v.wv]
    return np.mean(vecs, axis=0) if len(vecs) > 0 else np.zeros(200)

X_vec = np.array([get_w2v_vector(x.split()) for x in df['bersih']])
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(
    X_vec, y, test_size=0.2, random_state=42
)

scenarios = [
    {"name": "Linear C=1", "params": {"kernel": "linear", "C": 1}},
    {"name": "Linear C=10", "params": {"kernel": "linear", "C": 10}},
    {"name": "RBF C=1", "params": {"kernel": "rbf", "C": 1, "gamma": "scale"}},
    {"name": "RBF C=10", "params": {"kernel": "rbf", "C": 10, "gamma": "scale"}},
    {"name": "Poly C=1 d=3", "params": {"kernel": "poly", "C": 1, "degree": 3}},
]

best_acc = 0
best_pred = None
best_name = ""
best_model = None

for sc in scenarios:
    print("\n==============================")
    print(f"Running: {sc['name']}")
    print("==============================")

    model = SVC(**sc["params"])
    model.fit(X_train, y_train)

    pred = model.predict(X_test)
    acc = accuracy_score(y_test, pred)

    print(f"Akurasi: {acc:.4f}")

    if acc > best_acc:
        best_acc = acc
        best_pred = pred
        best_model = model
        best_name = sc["name"]

print("\n======================================")
print(" HASIL TERBAIK SVM + WORD2VEC")
print("======================================")
print(f"Skenario terbaik : {best_name}")
print(f"Akurasi terbaik  : {best_acc:.4f}\n")

print("Classification Report:")
print(classification_report(y_test, best_pred))



Running: Linear C=1
Akurasi: 0.4200

Running: Linear C=10
Akurasi: 0.4200

Running: RBF C=1
Akurasi: 0.5400

Running: RBF C=10
Akurasi: 0.4800

Running: Poly C=1 d=3
Akurasi: 0.3800

 HASIL TERBAIK SVM + WORD2VEC
Skenario terbaik : RBF C=1
Akurasi terbaik  : 0.5400

Classification Report:
              precision    recall  f1-score   support

    negative       0.53      0.38      0.44        21
     neutral       0.60      0.43      0.50         7
    positive       0.53      0.73      0.62        22

    accuracy                           0.54        50
   macro avg       0.56      0.51      0.52        50
weighted avg       0.54      0.54      0.53        50



<h1>3. Logistic Regression + TF-IDF</h1>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

X = df["bersih"].astype(str)
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

tfidf = TfidfVectorizer(max_features=8000, ngram_range=(1,2))
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

scenarios = [
    {"name": "C=0.1 L2", "params": {"C": 0.1, "penalty": "l2", "solver": "lbfgs", "max_iter": 500}},
    {"name": "C=0.5 L2", "params": {"C": 0.5, "penalty": "l2", "solver": "lbfgs", "max_iter": 500}},
    {"name": "C=1 L2", "params": {"C": 1, "penalty": "l2", "solver": "lbfgs", "max_iter": 500}},
    {"name": "C=2 L2", "params": {"C": 2, "penalty": "l2", "solver": "lbfgs", "max_iter": 500}},
    {"name": "C=5 L2", "params": {"C": 5, "penalty": "l2", "solver": "lbfgs", "max_iter": 500}},
    
    {"name": "C=1 L1", "params": {"C": 1, "penalty": "l1", "solver": "liblinear", "max_iter": 500}},
    {"name": "C=2 L1", "params": {"C": 2, "penalty": "l1", "solver": "liblinear", "max_iter": 500}},
    
    {"name": "SAGA ElasticNet 0.3", "params": {"C": 1, "penalty": "elasticnet", "l1_ratio": 0.3, "solver": "saga", "max_iter": 500}},
    {"name": "SAGA ElasticNet 0.5", "params": {"C": 1, "penalty": "elasticnet", "l1_ratio": 0.5, "solver": "saga", "max_iter": 500}},
    {"name": "SAGA ElasticNet 0.7", "params": {"C": 1, "penalty": "elasticnet", "l1_ratio": 0.7, "solver": "saga", "max_iter": 500}},
]

best_acc = 0
best_model = None
best_name = ""
best_pred = None


for sc in scenarios:
    print("\n=======================================")
    print(f"Running: {sc['name']}")
    print("=======================================")

    model = LogisticRegression(**sc["params"])
    model.fit(X_train_tfidf, y_train)

    pred = model.predict(X_test_tfidf)
    acc = accuracy_score(y_test, pred)

    print(f"Akurasi: {acc:.4f}")

    if acc > best_acc:
        best_acc = acc
        best_model = model
        best_pred = pred
        best_name = sc["name"]

print("\n=======================================")
print("HASIL TERBAIK Logistic Regression + TFIDF")
print("=======================================")
print(f"Skenario terbaik : {best_name}")
print(f"Akurasi terbaik  : {best_acc:.4f}\n")

print("Classification Report:")
print(classification_report(y_test, best_pred))



Running: C=0.1 L2
Akurasi: 0.5000

Running: C=0.5 L2
Akurasi: 0.5800

Running: C=1 L2
Akurasi: 0.6000

Running: C=2 L2
Akurasi: 0.6000

Running: C=5 L2
Akurasi: 0.5600

Running: C=1 L1
Akurasi: 0.4400

Running: C=2 L1
Akurasi: 0.4000

Running: SAGA ElasticNet 0.3
Akurasi: 0.4800

Running: SAGA ElasticNet 0.5
Akurasi: 0.4600

Running: SAGA ElasticNet 0.7
Akurasi: 0.4000

HASIL TERBAIK Logistic Regression + TFIDF
Skenario terbaik : C=1 L2
Akurasi terbaik  : 0.6000

Classification Report:
              precision    recall  f1-score   support

    negative       0.55      0.57      0.56        21
     neutral       0.60      0.43      0.50         7
    positive       0.65      0.68      0.67        22

    accuracy                           0.60        50
   macro avg       0.60      0.56      0.57        50
weighted avg       0.60      0.60      0.60        50



<h1>4. Logistic Regression + Word2Vec</h1>

In [ ]:
from gensim.models import Word2Vec
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report

sentences = df['bersih'].apply(lambda x: x.split())
w2v = Word2Vec(sentences, vector_size=200, min_count=2, window=5)

def get_w2v_vector(words):
    vecs = [w2v.wv[w] for w in words if w in w2v.wv]
    return np.mean(vecs, axis=0) if len(vecs) > 0 else np.zeros(200)

X_vec = np.array([get_w2v_vector(x.split()) for x in df['bersih']])
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, random_state=42)

param_grid = {
    "C": [0.01, 0.1, 1, 5, 10, 20],         
    "penalty": ["l1", "l2"],                
    "solver": ["liblinear", "saga"],       
    "max_iter": [300, 500]
}

logreg = LogisticRegression()

grid = GridSearchCV(
    logreg,
    param_grid,
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    verbose=2
)

grid.fit(X_train, y_train)

print("Best Parameters:", grid.best_params_)
print("\nBest Score:", grid.best_score_)

best_model = grid.best_estimator_
pred = best_model.predict(X_test)

print("\nClassification Report:")
print(classification_report(y_test, pred))


Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best Parameters: {'C': 20, 'max_iter': 300, 'penalty': 'l2', 'solver': 'saga'}

Best Score: 0.2694986007679506

Classification Report:
              precision    recall  f1-score   support

    negative       0.42      0.95      0.58        21
     neutral       0.00      0.00      0.00         7
    positive       0.00      0.00      0.00        22

    accuracy                           0.40        50
   macro avg       0.14      0.32      0.19        50
weighted avg       0.17      0.40      0.24        50



C:\Users\ASUS\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ASUS\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ASUS\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<h1>5. RNN + Word2Vec</h1>

In [ ]:
import numpy as np
import random
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score

sentences = df['bersih'].apply(lambda x: x.split())
w2v = Word2Vec(sentences, vector_size=200, min_count=2, window=5)

word_index = {word: i+1 for i, word in enumerate(w2v.wv.index_to_key)}

def to_seq(text):
    return [word_index[w] for w in text.split() if w in word_index]

X_seq = [to_seq(x) for x in df['bersih']]

le = LabelEncoder()
y_enc = le.fit_transform(df["label"])
y_cat = to_categorical(y_enc)

X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_seq, y_cat, test_size=0.2, random_state=42
)

embedding_matrix = np.zeros((len(word_index)+1, 200))
for w, i in word_index.items():
    embedding_matrix[i] = w2v.wv[w]

param_space = {
    "lstm_units": [64, 128, 256],
    "dropout": [0.2, 0.3, 0.4],
    "lr": [1e-3, 5e-4, 1e-4],
    "bidir": [True, False],
    "batch": [16, 32],
    "maxlen": [50, 60, 80]
}

def build_model(lstm_units, dropout, lr, bidir, maxlen):
    model = Sequential()

    model.add(
        Embedding(
            input_dim=len(word_index)+1,
            output_dim=200,
            weights=[embedding_matrix],
            input_length=maxlen,
            trainable=False
        )
    )

    if bidir:
        model.add(Bidirectional(LSTM(lstm_units)))
    else:
        model.add(LSTM(lstm_units))

    model.add(Dropout(dropout))

    model.add(Dense(64, activation="relu"))
    model.add(Dropout(dropout))

    model.add(Dense(len(le.classes_), activation="softmax"))

    model.compile(
        loss="categorical_crossentropy",
        optimizer=AdamW(learning_rate=lr),
        metrics=["accuracy"]
    )

    return model

best_acc = 0
best_params = None

N_TRIALS = 8  
for trial in range(N_TRIALS):
    print(f"\n===== TRIAL {trial+1}/{N_TRIALS} =====")

    params = {k: random.choice(v) for k, v in param_space.items()}
    print("PARAMETER:", params)

    X_train = pad_sequences(X_train_raw, maxlen=params["maxlen"])
    X_test = pad_sequences(X_test_raw, maxlen=params["maxlen"])

    model = build_model(
        lstm_units=params["lstm_units"],
        dropout=params["dropout"],
        lr=params["lr"],
        bidir=params["bidir"],
        maxlen=params["maxlen"]
    )

    es = EarlyStopping(
        monitor="val_loss", patience=3,
        restore_best_weights=True
    )

    model.fit(
        X_train, y_train,
        epochs=15,
        batch_size=params["batch"], 
        validation_split=0.1,
        callbacks=[es],
        verbose=0
    )

    pred = np.argmax(model.predict(X_test), axis=1)
    true = np.argmax(y_test, axis=1)
    acc = accuracy_score(true, pred)

    print("Akurasi:", acc)

    if acc > best_acc:
        best_acc = acc
        best_params = params


print("\n==============================")
print("TUNING SELESAI!")
print("BEST ACCURACY:", best_acc)
print("BEST PARAMETER:", best_params)
print("==============================")



===== TRIAL 1/8 =====
PARAMETER: {'lstm_units': 64, 'dropout': 0.3, 'lr': 0.0001, 'bidir': True, 'batch': 32, 'maxlen': 80}


C:\Users\ASUS\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 445ms/step
Akurasi: 0.44

===== TRIAL 2/8 =====
PARAMETER: {'lstm_units': 64, 'dropout': 0.3, 'lr': 0.0001, 'bidir': True, 'batch': 32, 'maxlen': 80}


C:\Users\ASUS\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 441ms/step
Akurasi: 0.44

===== TRIAL 3/8 =====
PARAMETER: {'lstm_units': 64, 'dropout': 0.4, 'lr': 0.001, 'bidir': False, 'batch': 32, 'maxlen': 60}


...

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 362ms/step
Akurasi: 0.44

===== TRIAL 7/8 =====
PARAMETER: {'lstm_units': 256, 'dropout': 0.4, 'lr': 0.001, 'bidir': True, 'batch': 16, 'maxlen': 80}


C:\Users\ASUS\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 266ms/step
Akurasi: 0.44

===== TRIAL 8/8 =====
PARAMETER: {'lstm_units': 64, 'dropout': 0.2, 'lr': 0.0001, 'bidir': False, 'batch': 32, 'maxlen': 60}


C:\Users\ASUS\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 497ms/step
Akurasi: 0.44

TUNING SELESAI!
BEST ACCURACY: 0.44
BEST PARAMETER: {'lstm_units': 64, 'dropout': 0.3, 'lr': 0.0001, 'bidir': True, 'batch': 32, 'maxlen': 80}


<h1>6. IndoBERT + TF-IDF</h1>

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import random

tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
bert = AutoModel.from_pretrained("indobenchmark/indobert-base-p1")

def get_bert_cls(text):
    tokens = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        output = bert(**tokens)
    return output.last_hidden_state[:,0,:].numpy().flatten()

print("Menghitung embedding BERT...")
bert_vec = np.array([get_bert_cls(t) for t in df['bersih']])

y = df["label"].values

param_space = {
    "tfidf_max": [3000, 5000, 7000, 10000],
    "ngram": [(1,1), (1,2)],
    "C": [0.1, 1, 2, 5, 10],
    "loss": ["hinge", "squared_hinge"]
}

best_acc = 0
best_params = None
best_report = None

N_TRIALS = 10
print("\n=== MULAI RANDOM SEARCH ===")

for trial in range(N_TRIALS):
    print(f"\nTRIAL {trial+1}/{N_TRIALS}")

    params = {
        key: random.choice(val)
        for key, val in param_space.items()
    }
    print("PARAMETER:", params)

    tfidf = TfidfVectorizer(
        max_features=params["tfidf_max"],
        ngram_range=params["ngram"]
    )

    tfidf_vec = tfidf.fit_transform(df["bersih"]).toarray()

    X = np.hstack([tfidf_vec, bert_vec])

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    model = LinearSVC(
        C=params["C"],
        loss=params["loss"],
        max_iter=5000
    )
    
    model.fit(X_train, y_train)
    pred = model.predict(X_test)

    acc = accuracy_score(y_test, pred)
    print("Akurasi:", acc)

    if acc > best_acc:
        best_acc = acc
        best_params = params
        best_report = classification_report(y_test, pred)

print("\n===================================")
print("=== HASIL AKHIR TUNING ===")
print("Best Accuracy:", best_acc)
print("Best Parameter:", best_params)
print("\n=== Classification Report Terbaik ===")
print(best_report)
print("===================================")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Menghitung embedding BERT...

=== MULAI RANDOM SEARCH ===

TRIAL 1/10
PARAMETER: {'tfidf_max': 7000, 'ngram': (1, 2), 'C': 10, 'loss': 'hinge'}


C:\Users\ASUS\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Akurasi: 0.5

TRIAL 2/10
PARAMETER: {'tfidf_max': 10000, 'ngram': (1, 2), 'C': 5, 'loss': 'squared_hinge'}


C:\Users\ASUS\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


...

Akurasi: 0.48

TRIAL 7/10
PARAMETER: {'tfidf_max': 7000, 'ngram': (1, 1), 'C': 0.1, 'loss': 'squared_hinge'}
Akurasi: 0.48

TRIAL 8/10
PARAMETER: {'tfidf_max': 10000, 'ngram': (1, 1), 'C': 1, 'loss': 'hinge'}


C:\Users\ASUS\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Akurasi: 0.5

TRIAL 9/10
PARAMETER: {'tfidf_max': 10000, 'ngram': (1, 2), 'C': 5, 'loss': 'squared_hinge'}


C:\Users\ASUS\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\ASUS\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\ASUS\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Akurasi: 0.48

TRIAL 10/10
PARAMETER: {'tfidf_max': 7000, 'ngram': (1, 2), 'C': 2, 'loss': 'hinge'}
Akurasi: 0.5

=== HASIL AKHIR TUNING ===
Best Accuracy: 0.5
Best Parameter: {'tfidf_max': 7000, 'ngram': (1, 2), 'C': 10, 'loss': 'hinge'}

=== Classification Report Terbaik ===
              precision    recall  f1-score   support

    negative       0.53      0.38      0.44        21
     neutral       0.42      0.71      0.53         7
    positive       0.52      0.55      0.53        22

    accuracy                           0.50        50
   macro avg       0.49      0.55      0.50        50
weighted avg       0.51      0.50      0.50        50

